In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import accuracy_score #Import accuracy model library.

In [ ]:
df = pd.read_csv('../input/bnp-paribas-cardif-claims-management/train.csv.zip')
df.shape

In [ ]:
df.head()

In [ ]:
df = df.drop("ID", axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [ ]:
train.shape

In [ ]:
train = train.fillna(train.median())
validate = validate.fillna(validate.median())

In [ ]:
from sklearn import preprocessing

for f in train.columns:
    if train[f].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.unique(list(train[f].values)  + list(validate[f].values)))
        train[f]   = lbl.transform(list(train[f].values))
        validate[f]  = lbl.transform(list(validate[f].values))


In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(train.select_dtypes(include=numerics).columns)
train = train[numerical_vars]
numerical_vars2 = list(validate.select_dtypes(include=numerics).columns)
validate = validate[numerical_vars]

In [ ]:
X_train = train.iloc[:,train.columns != 'target']
y_train = train.iloc[:, train.columns == 'target']
X_validate = validate.iloc[:,validate.columns != 'target']
y_validate = validate.iloc[:, validate.columns == 'target']

Feature Selection

In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest

univariate = f_classif(X_train, y_train)

univariate = pd.Series(univariate[1])
univariate.index = X_train.columns

In [ ]:
univariate.sort_values(ascending=False).plot.bar(figsize=(20, 8))

In [ ]:
# here I will select the top 20 features
sel_ = SelectKBest(f_classif, k=20).fit(X_train, y_train)
X_train.columns[sel_.get_support()]

In [ ]:
columns = ['v4', 'v10', 'v12', 'v14', 'v21', 'v31', 'v34', 'v38', 'v47', 'v50',
       'v56', 'v62', 'v66', 'v72', 'v110', 'v113', 'v114', 'v119', 'v123',
       'v129']
X_train = X_train[columns]
X_validate = X_validate[columns]

Modelling

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_validate)
accuracy_score(y_validate, y_pred)